<a href="https://colab.research.google.com/github/Shrijeet16/kaggle-inclass-Competition/blob/master/powerplant_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggleDownloader

In [2]:
from kaggleDownloader import get_dataset

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sj161199","key":"9e960ddbb01322f1478dfef7a8a5c372"}'}

In [4]:
get_dataset('kaggle competitions download -c power-plant-classification')


Download completed..
Unzipping the zip files

Zip Files unzipped

 Directory contains the following files :  ['.config', 'kaggle.json', 'landsat_power_plants_train', 'naip_power_plants_train', 'naip_power_plants_test', 'training_labels.csv', 'power-plant-classification.zip', 'sample_submission.csv', 'landsat_power_plants_test', 'sample_data']

Remove zip files ? (yes/no) :yes

 Directory contains the following files :  ['.config', 'kaggle.json', 'landsat_power_plants_train', 'naip_power_plants_train', 'naip_power_plants_test', 'training_labels.csv', 'sample_submission.csv', 'landsat_power_plants_test', 'sample_data']


In [5]:
from tensorflow.keras.applications import vgg16
import keras
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
                rescale = 1./255,
                rotation_range = 30,
                width_shift_range = 20,
                height_shift_range = 20,
                brightness_range = (0.4,0.6),
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True,
                vertical_flip = True)


In [7]:
train_data_dir = 'landsat_power_plants_train/train/'

In [8]:
image_height , image_width = 75 , 75
batch_size = 16

In [9]:
train_generator = train_datagen.flow_from_directory(
                  train_data_dir,
                  target_size = (image_height, image_width),
                  class_mode = 'categorical',
                  batch_size = batch_size)


Found 2841 images belonging to 10 classes.


In [10]:
test_datagen = ImageDataGenerator(
                rescale = 1./255)

In [11]:
valid_data_dir = 'landsat_power_plants_train/valid/'
test_data_dir = 'landsat_power_plants_test/'

In [12]:
validation_generator = test_datagen.flow_from_directory(
                        valid_data_dir,
                        target_size = (image_height, image_width),
                        class_mode = 'categorical',
                        batch_size = batch_size)


Found 693 images belonging to 10 classes.


In [41]:
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))

In [42]:
for layer in vgg_conv.layers:
  if layer.name == 'block5_conv1':
    break
  layer.trainable = False


vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 75, 75, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0     

In [43]:
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.datasets import cifar10
from keras.utils import to_categorical

In [46]:
last = vgg_conv.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(80, activation='relu', name = 'fc2')(x)
x = Dropout(0.3)(x)
x = Dense(10, activation='softmax', name='predictions')(x)

model = Model(vgg_conv.input, x)
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 75, 75, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)     

In [47]:
model.compile(optimizer = Adam(learning_rate=0.001) , 
              loss = 'categorical_crossentropy' , metrics = ['accuracy'])


In [48]:
no_train_examples = train_generator.samples
no_valid_examples = validation_generator.samples
epochs = 50
batch_size = 16

In [49]:
model.fit_generator(
    train_generator,
    steps_per_epoch = no_train_examples//batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = no_valid_examples//batch_size
)

Epoch 1/50
177/177 [==============================] - 9s 51ms/step - loss: 2.1069 - accuracy: 0.2329 - val_loss: 2.0590 - val_accuracy: 0.1701
Epoch 2/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0250 - accuracy: 0.2372 - val_loss: 1.9674 - val_accuracy: 0.2689
Epoch 3/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0069 - accuracy: 0.2619 - val_loss: 1.9770 - val_accuracy: 0.2747
Epoch 4/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0076 - accuracy: 0.2531 - val_loss: 1.9701 - val_accuracy: 0.2747
Epoch 5/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0046 - accuracy: 0.2570 - val_loss: 2.1391 - val_accuracy: 0.2805
Epoch 6/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0147 - accuracy: 0.2743 - val_loss: 2.0045 - val_accuracy: 0.2703
Epoch 7/50
177/177 [==============================] - 9s 50ms/step - loss: 2.0062 - accuracy: 0.2662 - val_loss: 2.0041 - val_accuracy: 0.2762